In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from darkflow.net.build import TFNet
import cv2
import tensorflow as tf
import os
import math
import time

In [2]:
options2 = {
    'gpu': 1.0,
    'model': 'cfg/yolov2-tiny-hand.cfg',
    'load': 56800,
    'threshold': 0.1,
    'backup': 'ckpt'
}
tfnet2 = TFNet(options2)
tfnet2.load_from_ckpt()

Parsing cfg/yolov2-tiny-hand.cfg
Loading None ...
Finished in 0.0009982585906982422s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
Instructions for updating:
Colocations handled automatically by placer.
       |        | input                            | (?, 416, 416, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 26, 26, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 2

In [13]:
direc = os.path.join('new_data', 'egohands', 'validation')
image_names = os.listdir(direc)
image_annots = os.listdir(direc + '_annotations')


imgs = []
num_detected = 0

orig_height = 416
orig_width = 416

model_time = 0
total_time = 0
total_no_write_time = 0

for i, img_name in enumerate(image_names):
    start_time = time.time()
    img = cv2.imread(os.path.join(direc, img_name))
    height, width, _ = img.shape
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    model_start_time = time.time()
    results = tfnet2.return_predict(img)
    model_end_time = time.time()
    pred_time = model_end_time - model_start_time
    model_time += pred_time
    if len(results) != 0:
        num_detected += 1
        for res in results:
#             print(res)
            top_left = (res['topleft']['x'], res['topleft']['y'])
            bottom_right = (res['bottomright']['x'], res['bottomright']['y'])
            label = res['label']
            confidence = res['confidence']
            top_left_conf = (res['topleft']['x'], res['topleft']['y'] + 10)
            img = cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 7)
            img = cv2.putText(img, label, top_left, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
            img = cv2.putText(img, str(confidence), top_left_conf, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    imgs.append(img)
    no_write_end_time = time.time()
    pred_time = no_write_end_time - start_time
    total_no_write_time += pred_time
    cv2.imwrite('output/img{}.jpg'.format(i), img)
    end_time = time.time()
    pred_time = end_time - start_time
    total_time += pred_time


num_images = len(image_names)
height, width, channels = img.shape
print('Image resolution: {}x{}'.format(height, width))
print('Num images found: {} out of {} => {} in {} seconds'.format(num_detected, num_images, num_detected / num_images, pred_time))
print('Total time: {}'.format(total_time))
avg_model_time = model_time / num_images
print('Avg model time: {} or {} fps'.format(avg_model_time, 1 / avg_model_time))
avg_no_write_time = total_no_write_time / num_images
print('Avg no write time: {} or {} fps'.format(avg_no_write_time, 1 / avg_no_write_time))
avg_time = total_time / num_images
print('Avg time: {} or {} fps'.format(avg_time, 1 / avg_time))

Image resolution: 480x640
Num images found: 383 out of 383 => 1.0 in 0.07280516624450684 seconds
Total time: 27.80911898612976
Avg model time: 0.051503989466176646 or 19.415971662869207 fps
Avg no write time: 0.06184831425041816 or 16.168589429149055 fps
Avg time: 0.07260866576012992 or 13.772460759761119 fps


In [4]:
  
# plotImages(imgs)